## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [1]:
import numpy as np
import pandas as pd

from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split

# 分類問題：LogisticRegression；迴歸問題：LinearRegression
from sklearn.linear_model import LogisticRegression, LinearRegression 

# 決策樹：
# 分類問題：DecisionTreeClassifier；迴歸問題：DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

# 隨機森林
# 分類問題：RandomForestClassifier；迴歸問題：RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

# 梯度提升機
# 分類問題：GradientBoostingClassifier；迴歸問題：GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor

# 評估模型
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

### 梯度提升機（分類問題）

In [2]:
# 讀取資料集
wine = datasets.load_wine( ) 

# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split( wine.data, wine.target, test_size = 0.35, random_state = 4 )

In [3]:
# 建立梯度提升機模型
GBC_1 = GradientBoostingClassifier( n_estimators = 5, learning_rate = 0.01 )
# n_estimators ( default = 100 )：最大的迭代次數，即模型的數量 ( 過大容易產生過擬合 )
# learning_rate ( default = 0.1 )：每個模型的權重數，較小的 learning_rate 會需要較多的迭代次數(n_estimators)
# 註：調參數時， n_estimators 與  learning_rate 會一起調整！
# loss ( default = ’deviance’ )：分類模型的損失函數有 'deviance' 及 'exponential' 兩種；若選用 'exponential' ，則是套用 Adaboost 演算法。
#                                          一般建議使用預設，它對二元或多元分類有較佳的優化效果。


# 訓練模型
GBC_1.fit( x_train, y_train )

# 預測測試集
y_pred = GBC_1.predict( x_test )

# 評估模型
acc = metrics.accuracy_score( y_test, y_pred )
print( 'Acuuracy : ', acc,  '\n' )

print( '預測值 ：' + str( y_pred ) )
print( '實際值 ：' + str( y_test ) + '\n' )

Acuuracy :  0.6507936507936508 

預測值 ：[1 1 0 0 1 1 0 1 0 1 1 0 1 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 1 0
 0 0 1 1 0 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1 1 1 0 1 1]
實際值 ：[2 2 0 0 1 2 0 1 0 1 1 0 2 2 0 1 0 1 1 2 1 2 1 2 0 2 1 1 2 2 0 1 0 1 2 2 0
 0 0 2 2 0 0 1 1 0 1 2 0 2 1 1 1 0 0 1 1 1 2 2 0 2 1]



In [4]:
# 檢視模型的參數
GBC_1

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.01, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=5,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

### Hyperparameter Optimisation

##### 隨機搜尋 ( Random Search )：

函數  RandomizedSearchCV( )
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html 
  
搭配相關函數
- np.arange : https://docs.scipy.org/doc/numpy/reference/generated/numpy.arange.html  
- np.linspace : https://docs.scipy.org/doc/numpy/reference/generated/numpy.linspace.html

  
相關文獻：
- https://towardsdatascience.com/algorithms-for-hyperparameter-optimisation-in-python-edda4bdb167


In [5]:
# 設定搜尋引數搜尋範圍
param_dist = { 'n_estimators' : np.arange( 20, 120, 4 ), 
                     'learning_rate' : np.linspace( 0.01, 2, 20 ), 
                     'max_depth' : np.arange( 1, 4 ) 
                    }

param_dist

{'n_estimators': array([ 20,  24,  28,  32,  36,  40,  44,  48,  52,  56,  60,  64,  68,
         72,  76,  80,  84,  88,  92,  96, 100, 104, 108, 112, 116]),
 'learning_rate': array([0.01      , 0.11473684, 0.21947368, 0.32421053, 0.42894737,
        0.53368421, 0.63842105, 0.74315789, 0.84789474, 0.95263158,
        1.05736842, 1.16210526, 1.26684211, 1.37157895, 1.47631579,
        1.58105263, 1.68578947, 1.79052632, 1.89526316, 2.        ]),
 'max_depth': array([1, 2, 3])}

In [6]:
from sklearn.model_selection import RandomizedSearchCV

# 建立梯度提升機模型
GBC = GradientBoostingClassifier( )

# 隨機搜尋函數設定
rs_random = RandomizedSearchCV( estimator = GBC, param_distributions = param_dist, cv = 5, n_iter = 100, n_jobs = -1 )
# n_iter：訓練次數。次數越大可獲得的引數精度越大，但是搜尋的時間也相對增加。
# n_jobs = -1：使用所有的CPU進行訓練，預設為1，使用1個CPU。


# 搜尋模型的最佳參數
rs_random.fit( x_train, y_train )

# 最佳結果與最佳參數
print( 'Best Accuracy : %f using %s' % ( rs_random.best_score_ , rs_random.best_params_ ) )

Best Accuracy : 0.982609 using {'n_estimators': 48, 'max_depth': 2, 'learning_rate': 1.371578947368421}


/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [7]:
# 使用最佳參數重新建立模型
GBC_best_param = GradientBoostingClassifier( n_estimators = rs_random.best_params_[ 'n_estimators' ],
                                                                   max_depth = rs_random.best_params_[ 'max_depth' ],
                                                                   learning_rate = rs_random.best_params_[ 'learning_rate' ] )
                                                             
# 訓練模型
GBC_best_param.fit( x_train, y_train )

# 預測測試集
y_pred_best_param = GBC_best_param.predict( x_test )

# 評估模型
accy_best_param = metrics.accuracy_score( y_test, y_pred_best_param )
print( 'Acuuracy : ', accy_best_param,  '\n' )

print( '預測值 ：' + str( y_pred_best_param ) )
print( '實際值 ：' + str( y_test ) + '\n' )

Acuuracy :  0.9682539682539683 

預測值 ：[2 2 0 0 1 2 0 1 0 1 1 0 2 2 0 1 0 1 1 2 1 2 1 2 0 2 1 1 2 2 0 1 0 1 2 2 0
 0 0 2 2 0 0 2 1 0 1 2 0 2 1 1 1 0 0 1 2 1 2 2 0 2 1]
實際值 ：[2 2 0 0 1 2 0 1 0 1 1 0 2 2 0 1 0 1 1 2 1 2 1 2 0 2 1 1 2 2 0 1 0 1 2 2 0
 0 0 2 2 0 0 1 1 0 1 2 0 2 1 1 1 0 0 1 1 1 2 2 0 2 1]

